# `Component` walkthrough
This notebook demonstrates how components work together in NeuroMANCER, and how data flows through the computational graph assembled by the `Problem` class. We'll demonstrate by building a full neural state space model for partially-observable dynamical systems.

In [1]:
import torch
import torch.nn.functional as F
from neuromancer.estimators import MLPEstimator
from neuromancer.dynamics import block_model, BlockSSM
from neuromancer.problem import Problem
from neuromancer.constraint import Loss
from neuromancer.blocks import MLP
from neuromancer.plot import plot_model_graph
import slim

We begin constructing our neural SSM by creating a latent state estimator to predict initial conditions from past observations. To do this, we specify the dimensionality of both our observables and the latent state; in this case, we choose 10 and 20, respectively.

As you will see after running the cell below, each component has a handy string representation that indicates the name of the component, its input variables, and its outputs. Notice that the outputs of the component are tagged with the name of the component that produced them; this is used to prevent name collisions in the computational graph, and will become important in the next step.

In [2]:
estim = MLPEstimator(
    {"Yp": (10,), "x0": (20,)},
    nsteps=2,
    name="estim"
)
estim

estim(Yp) -> x0_estim, reg_error_estim

Next, we define our state space model. The SSM component will take the output of the estimator as its initial condition `x0`. However, recall that components tag their outputs with their name. If we look at the default input keys of the `BlockSSM` class, we'll notice a slight problem:

In [3]:
BlockSSM.DEFAULT_INPUT_KEYS

['x0', 'Yf']

The canonical name for the initial state variable in `BlockSSM`s is named `x0`, not `x0_estim`. Because of this, we need to remap the estimator's output `x0_estim` to `x0`. To do this, we hand a dictionary to the `input_keys` parameter of the `block_model` function which maps the tagged variable to the canonical name used by the dynamics model. The following cell uses the named constructor `block_model` to generate a Block Nonlinear neural SSM:

In [5]:
dynamics = block_model(
    "blocknlin",
    {"x0": (20,), "Yf": (10,), "Uf": (2,)},
    slim.Linear,
    MLP,
    bias=False,
    input_key_map={"x0": "x0_estim", "Uf": "Uf_renamed"},
    name="dynamics"
)

Now we have both model components ready to compose with a `Problem` instance. However, before we do that, let's pick apart how data flows through the computational graph formed by these components when composed in a `Problem`.

First, we'll create a `DataDict` containing the constant inputs required by each component (`Yp`, `Yf`, and `Uf` for the dynamics model; and `Yp` for the estimator).

In [6]:
data = {
    "Yp": torch.rand(2, 10, 10),
    "Yf": torch.rand(2, 10, 10),
    "Uf_renamed": torch.rand(2, 10, 2),
}

Next, let's push the data through the estimator to see what we receive as output (note that we combine the data and estimator output to retain the constant inputs used by the dynamics model):

In [7]:
output = {**data, **estim(data)}
output.keys()

dict_keys(['Yp', 'Yf', 'Uf_renamed', 'x0_estim', 'reg_error_estim'])

As expected, we obtain our estimated initial state `x0_estim` alongside a `reg_error_estim` term measuring the regularization error incurred by any structured linear maps in the component (in this case there are none).

Now let's take the output of the estimator and push it through the SSM:

In [8]:
output = {**output, **dynamics(output)}
output.keys()

dict_keys(['Yp', 'Yf', 'Uf_renamed', 'x0_estim', 'reg_error_estim', 'fU_dynamics', 'X_pred_dynamics', 'Y_pred_dynamics', 'reg_error_dynamics'])

As we can see, the dynamics model correctly handles the `x0_estim` variable; internally, the variable is automatically renamed to its canonical name. This capability allows users to combine components in arbitrary ways.

We'll next create some objectives and constraints to demonstrate how these interact with components in a `Problem` instance; we define the inputs to each objective and constraint by providing a list of keys which can reference either the input data or the output of any component in the overall model.

In [9]:
reference_loss = Loss(["Y_pred_dynamics", "Yf"], F.mse_loss, name="reference_loss")
estimator_loss = Loss(["X_pred_dynamics", "x0_estim"], lambda x, y: F.mse_loss(x[-1, :-1, :], y[1:]), name="estimator_loss")
bounds_constraint = Loss(["Y_pred_dynamics"], lambda x: F.relu(0.5 - x).mean(), name="bounds_constraint")

At last, let's put together a `Problem` class to combine everything. Like `Component`s, when we instantiate a `Problem` we can inspect its string representation to get an overview of all the constructs in the model and see how they are put together.

In [10]:
objectives = [reference_loss, estimator_loss]
constraints = [bounds_constraint]
trainable_components = [estim, dynamics]
model = Problem(objectives, constraints, trainable_components)
model

### MODEL SUMMARY ###

COMPONENTS:
  estim(Yp) -> x0_estim, reg_error_estim
  dynamics(Yf, x0_estim, Uf_renamed) -> fU_dynamics, X_pred_dynamics, Y_pred_dynamics, reg_error_dynamics

CONSTRAINTS:
  bounds_constraint(Y_pred_dynamics) -> <function <lambda> at 0x0000024776ACEF78> * 1.0

OBJECTIVES:
  reference_loss(Y_pred_dynamics, Yf) -> <function mse_loss at 0x000002475C270708> * 1.0
  estimator_loss(X_pred_dynamics, x0_estim) -> <function <lambda> at 0x0000024776AB58B8> * 1.0

With our `Problem` created, we can now push the data dictionary we previously defined through it to receive the outputs of each component and the values of each objective and constraint we specified. Note that we wrap the data into a `DataDict` and add a `name` attribute; like the attribute used in `Component`s, this is used to prevent name collisions between variables generated by the use of different data splits during training and validation.

In [11]:
data["name"] = "test"
output = model(data)
output.keys()


dict_keys(['test_Yp', 'test_Yf', 'test_Uf_renamed', 'test_name', 'test_x0_estim', 'test_reg_error_estim', 'test_fU_dynamics', 'test_X_pred_dynamics', 'test_Y_pred_dynamics', 'test_reg_error_dynamics', 'test_reference_loss', 'test_estimator_loss', 'test_bounds_constraint', 'test_loss'])

And that's all there is to it. The model can now be passed to a `Trainer` along with a `Dataset` instance to train and validate the model.